In [ ]:
!pip install langchain

In [ ]:
!pip install -U langchain-community


#Data Loading Techniques

In [6]:
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders.csv_loader import CSVLoader


In [13]:
from google.colab import files
uploaded =files.upload()

Saving hazard_detection_dataset_2000.csv to hazard_detection_dataset_2000.csv


In [14]:
loader = CSVLoader(file_path="hazard_detection_dataset_2000.csv")
data = loader.load()

In [17]:
data[0].metadata

{'source': 'hazard_detection_dataset_2000.csv', 'row': 0}

In [18]:
data[0].page_content

'dht_temperature: 48.17\nhumidity: 72.18\nbmp_temperature: 33.27\npressure: 1001.82\naltitude: 15.98\nmq4_analog_value: 475\nmq4_ppm_estimate: 124.24\nmq4_digital_output: 1\naccel_x: 7.75\naccel_y: -8.37\naccel_z: 7.21\nvibration: 1\nsound_detected: 0\nflame_detected: 0\nsos: 0\nhazard: Electrical Fault'

In [19]:
  #installing necessary libraries, libmagic is used for file type detection
!pip3 install unstructured libmagic python-magic python-magic-bin

  Preparing metadata (setup.py) ... done
ERROR: Ignored the following versions that require a different python version: 0.12.0 Requires-Python >=3.9.0,<3.12; 0.12.2 Requires-Python >=3.9.0,<3.12; 0.12.3 Requires-Python >=3.9.0,<3.12; 0.12.4 Requires-Python >=3.9.0,<3.12; 0.12.5 Requires-Python >=3.9.0,<3.12; 0.12.6 Requires-Python >=3.9.0,<3.12; 0.13.0 Requires-Python <3.12,>=3.9.0; 0.13.1 Requires-Python <3.12,>=3.9.0; 0.13.2 Requires-Python <3.12,>=3.9.0; 0.13.3 Requires-Python <3.12,>=3.9.0; 0.13.4 Requires-Python <3.12,>=3.9.0; 0.13.5 Requires-Python <3.12,>=3.9.0; 0.13.6 Requires-Python <3.12,>=3.9.0; 0.13.7 Requires-Python <3.12,>=3.9.0; 0.14.0 Requires-Python <3.12,>=3.9.0
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin


In [ ]:
!pip install unstructured

In [28]:
from langchain_community.document_loaders import UnstructuredURLLoader
loader = UnstructuredURLLoader(
    urls = [
        "https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html",
        "https://www.moneycontrol.com/news/business/markets/market-corrects-post-rbi-ups-inflation-forecast-icrr-bet-on-these-top-10-rate-sensitive-stocks-ideas-11142611.html"
    ]
)

In [29]:
data=loader.load()

In [35]:
data[0].metadata

{'source': 'https://www.moneycontrol.com/news/business/banks/hdfc-bank-re-appoints-sanmoy-chakrabarti-as-chief-risk-officer-11259771.html'}

In [36]:
len(data)


2

#Text Splitting

LLM's have token limits. Hence we need to split the text which can be large into small chunks so that each chunk size is under the token limit. There are various text splitter classes in langchain that allows us to do this.

In [38]:
text=''' Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin; lit. 'The Advancing Giant') is a Japanese manga series written and illustrated by Hajime Isayama. Set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans, the story follows Eren Yeager, an adolescent boy who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha's monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.

An anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episode third season was broadcast in two parts, with the first 12 episodes airing from July to October 2018 and the last 10 episodes airing from April to July 2019. A fourth and final season premiered in December 2020, airing 16 episodes in its first part. A second part consisting of 12 episodes aired from January to April 2022, and the third and fourth parts aired in two specials; the first premiered in March 2023, and the second premiered in November 2023.

Attack on Titan has been met with widespread critical and commercial success. By November 2023, the manga had over 140 million copies in circulation, making it one of the best-selling manga series of all time. It has won several awards, including the Kodansha Manga Award, the Attilio Micheluzzi Award, and the Harvey Award.

'''

In [39]:
text

" Attack on Titan (Japanese: 進撃の巨人, Hepburn: Shingeki no Kyojin; lit.\u2009'The Advancing Giant') is a Japanese manga series written and illustrated by Hajime Isayama. Set in a world where humanity is forced to live in cities surrounded by three enormous walls that protect them from gigantic man-eating humanoids referred to as Titans, the story follows Eren Yeager, an adolescent boy who vows to exterminate the Titans after they bring about the destruction of his hometown and the death of his mother. It was serialized in Kodansha's monthly magazine Bessatsu Shōnen Magazine from September 2009 to April 2021, with its chapters collected in 34 tankōbon volumes.\n\nAn anime television series was produced by Wit Studio (seasons 1–3) and MAPPA (season 4). A 25-episode first season was broadcast from April to September 2013, followed by a 12-episode second season broadcast from April to June 2017. A 22-episode third season was broadcast in two parts, with the first 12 episodes airing from July

In [40]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size=200,
    chunk_overlap=0
)

In [41]:
splitted_data= splitter.split_text(text)

In [42]:
for i in splitted_data:
  print(len(i))

658
702
324


so this one has a problem, that it only splits based on the delimter we pass, so it might not be able to split even when the chunk passes the chunk size limit

to fix this problem we can use recursive text splitter

In [43]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    separators = ["\n\n", "\n", " "],  # List of separators based on requirement (defaults to ["\n\n", "\n", " "])
    chunk_size = 200,  # size of each chunk created
    chunk_overlap  = 0,  # size of  overlap between chunks in order to maintain the context
    length_function = len  # Function to calculate size, currently we are using "len" which denotes length of string however you can pass any token counter)
)

In [45]:
data =r_splitter.split_text(text)

In [48]:
for i in data:
  print(len(i))

193
196
195
71
198
199
196
106
199
124


how it works:
so it tries to split iteratively with the next delimter when a specific chunk size is greater than the limit,...
